# NVIDIA Trellis NIM GPU Workflow
This notebook submits image-to-3D jobs to the NVIDIA Trellis NIM container and polls for completion.
Configure your environment according to [`docs/trellis_nim_setup.md`](../docs/trellis_nim_setup.md) before running.


## 1. Install runtime dependencies
Run the following cell once per environment to ensure the required Python packages are installed.


In [ ]:
%pip install -q requests python-dotenv pillow tqdm


## 2. Configure environment
Set the base URL for the Trellis NIM container (default assumes it is available at `http://localhost:8000`).
You can optionally provide an API key if your proxy requires one.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

NIM_BASE_URL = os.environ.get('NIM_BASE_URL', 'http://localhost:8000')
NIM_JOB_PATH = os.environ.get('NIM_JOB_PATH', '/v1/images-to-3d')
NIM_API_KEY = os.environ.get('NIM_API_KEY')

BASE_URL = NIM_BASE_URL.rstrip('/')
JOB_ROOT = f"{BASE_URL}{NIM_JOB_PATH}".rstrip('/')
SUBMIT_ENDPOINT = f"{JOB_ROOT}/jobs"
STATUS_ENDPOINT = f"{JOB_ROOT}/jobs/{{job_id}}"

HEADERS = {'Content-Type': 'application/json'}
if NIM_API_KEY:
    HEADERS['Authorization'] = f'Bearer {NIM_API_KEY}'

print('Submit endpoint:', SUBMIT_ENDPOINT)
print('Status endpoint:', STATUS_ENDPOINT)


## 3. Select input images
Provide one or more paths to local image files. They will be Base64-encoded before being sent to NIM.


In [ ]:
from pathlib import Path
from typing import Sequence
import base64

IMAGE_PATHS = [
    # Example: Path('/data/chair_front.png'),
]

def encode_images(paths: Sequence[Path]) -> list[str]:
    encoded = []
    for path in paths:
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(path)
        with path.open('rb') as f:
            encoded.append(base64.b64encode(f.read()).decode('utf-8'))
    return encoded


## 4. Submit the Trellis generation job


In [ ]:
import json
import requests

if not IMAGE_PATHS:
    raise ValueError('Populate IMAGE_PATHS with at least one image path before submitting.')

payload = {
    'images': encode_images(IMAGE_PATHS),
    'meshFormat': 'glb',
    'textureFormat': 'png',
}

response = requests.post(SUBMIT_ENDPOINT, headers=HEADERS, json=payload, timeout=120)
response.raise_for_status()
submission = response.json()
submission


## 5. Poll for job completion


In [ ]:
import time
from tqdm.auto import tqdm

def poll_job(job_id: str, interval: float = 5.0, timeout: float = 900.0):
    start = time.time()
    with tqdm(total=int(timeout // interval), desc='Polling NIM job') as progress:
        while True:
            if time.time() - start > timeout:
                raise TimeoutError('Timed out waiting for NIM job to finish.')
            status_response = requests.get(
                STATUS_ENDPOINT.format(job_id=job_id),
                headers=HEADERS,
                timeout=120,
            )
            status_response.raise_for_status()
            data = status_response.json()
            progress.set_postfix_str(data.get('status', 'unknown'))
            if data.get('message'):
                progress.set_description_str(data['message'][:48])
            if data.get('status') in {'succeeded', 'failed'}:
                return data
            time.sleep(interval)
            progress.update(1)

result = poll_job(submission['jobId'])
result


## 6. Download returned assets (optional)


In [ ]:
from pathlib import Path

output_dir = Path('nim_outputs')
output_dir.mkdir(exist_ok=True)

if result.get('assets'):
    for asset in result['assets']:
        asset_url = asset['url']
        extension = Path(asset_url).suffix or '.bin'
        target = output_dir / f"{submission['jobId']}_{asset['type']}{extension}"
        asset_response = requests.get(asset_url, timeout=300)
        asset_response.raise_for_status()
        target.write_bytes(asset_response.content)
        print('Saved', target)
else:
    print('No assets returned by the job.')
